<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [809]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')
# основные библиотеки|
import pandas as pd

# дополнительные библиотеки
import requests
import xml.etree.ElementTree as ET
import datetime
import re

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [810]:
# ВАШ КОД ЗДЕСЬ
df = pd.read_csv('df2021-08-03.csv')

In [811]:
df.dtypes

vacancy         object
url             object
created         object
has_test          bool
salary_from    float64
salary_to      float64
currency        object
experience      object
schedule        object
skills          object
employer        object
area            object
description     object
dtype: object

In [812]:
# приведём столбец с датами к формату даты
df['created'] = pd.to_datetime(df['created'])
df['created'] = df['created'].dt.strftime('%Y-%m-%d')

In [813]:
# зарплату к числовому виду
df['salary_from'] = df['salary_from'].astype(float)
df['salary_to'] = df['salary_to'].astype(float)

In [814]:
df.isna().sum()

vacancy           0
url               0
created           0
has_test          0
salary_from    1162
salary_to      1267
currency       1087
experience        0
schedule          0
skills          210
employer          0
area              0
description       0
dtype: int64

### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [815]:
# Выберем вариант 1 - на текущую дату.

In [816]:
# какие валюты представлены
df.currency.unique()

array(['RUR', nan, 'EUR', 'USD'], dtype=object)

In [817]:
# ВАШ КОД ЗДЕСЬ

# выполним запрос к апи ЦБ РФ
# возьмём курс $ на сегодняшнюю дату
usd_rate = float(
                ET.fromstring(requests.get('http://www.cbr.ru/scripts/XML_daily.asp').text)
                .find("./Valute[CharCode='USD']/Value")
                .text.replace(",","."))


# выполним запрос к апи ЦБ РФ
# возьмём курс € на сегодняшнюю дату
euro_rate = float(
                ET.fromstring(requests.get('http://www.cbr.ru/scripts/XML_daily.asp').text)
                .find("./Valute[CharCode='EUR']/Value")
                .text.replace(",","."))


# приведём валюты к рублю
for i in range(len(df)):
    
    if df['currency'][i] == "USD":
        
        df['currency'][i] = "RUR"
        
        df['salary_to'][i]  = df['salary_to'][i]*usd_rate
        df['salary_from'][i]  = df['salary_from'][i]*usd_rate
        
    elif df['currency'][i] == "EUR":
        
        df['currency'][i] = "RUR"
        
        df['salary_to'][i]  = df['salary_to'][i]*euro_rate
        df['salary_from'][i]  = df['salary_from'][i]*euro_rate
        
    else:
        pass

In [818]:
# данные приведены к рублям
df.currency.unique()

array(['RUR', nan], dtype=object)

In [834]:
# сохраним полученный датафрейм, чтобы поработать с ним в PowerBI
df.to_csv('test_bi.csv', index=False)

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [820]:
# посмотрим возможные интервалы опыта
df.experience.unique()

array(['От 1 года до 3 лет', 'От 3 до 6 лет', 'Нет опыта', 'Более 6 лет'],
      dtype=object)

In [845]:
highest_salary = df
# примем, что фактическая заработная плата - среднее зарплатной вилки
highest_salary['salary'] = (highest_salary['salary_from'] + highest_salary['salary_to'])/2

In [846]:
highest_salary = highest_salary.query('(experience == "От 3 до 6 лет") and (has_test == False)')\
    .sort_values(by = 'salary', ascending = False)

In [847]:
#  вакансия с самой высокой зарплатной вилкой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.
highest_salary.head(1)

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description,salary
191,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02,False,355253.2,532879.8,RUR,От 3 до 6 лет,Удаленная работа,Python;Django Framework;Git;SQL;PostgreSQL;REST;,Top Remote Talent,Москва,A Dubai based startup in the PropTech industry...,444066.5


#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [831]:
# ВАШ КОД ЗДЕСЬ

# разобьём списки со скиллами на подстроки
df['skills'] = df['skills'].apply(lambda x: str(x).split(';'))

# добавим все значения, не исключая повторение, в один список
skill = []
for x in range(len(df)):
    skill.extend(df['skills'][x])
    
# создадим отдельный df для возможного использования полученных данных вспоследствии
skill_count = {}
for x in skill:
    skill_count[x] = skill.count(x)
skill_df = pd.DataFrame(list(skill_count.items()), columns=['Skill', 'Count']).query("Skill != ''")

# топ 30 навыков по всем вакансиям по числу упоминаний
skill_df = skill_df.sort_values('Count', ascending = False).head(30)
skill_df

,Skill,Count
0,Python,1442
5,Git,586
25,SQL,501
1,PostgreSQL,499
9,Django Framework,467
2,Linux,463
15,nan,210
3,Flask,177
30,Docker,172
19,JavaScript,149


In [833]:
# сохраним полученный датафрейм, чтобы поработать с ним в PowerBI
skill_df.to_csv('skills_bi.csv', index=False)

#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [825]:
# ВАШ КОД ЗДЕСЬ

# вакансии тестировщиков
# примем, что SDET тоже попадает по вакансии тестировщиков
# эдакий прокачанный qa
qa_df = df[df['vacancy'].str.contains(r'qa|test|тест|uat|assurance', case=False)]

# подсчитаем без учёта отсутствия данных по зарплате
qa_df['salary'] = (qa_df['salary_from'] + qa_df['salary_to'])/2
qa_df.groupby('area', as_index = False).agg({'salary_from':'mean','salary_to':'mean', 'salary':'mean' })\
    .sort_values(by = 'salary_from', ascending = False)

,area,salary_from,salary_to,salary
6,Москва,171826.655556,183750.000000,147500.0
16,Тамбов,150000.000000,250000.000000,200000.0
12,Санкт-Петербург,148000.000000,209285.714286,175000.0
4,Казань,90000.000000,NaN,NaN
0,Брянск,NaN,NaN,NaN
1,Дзержинск (Нижегородская область),NaN,NaN,NaN
2,Димитровград,NaN,NaN,NaN
3,Екатеринбург,NaN,NaN,NaN
5,Краснодар,NaN,NaN,NaN
7,Нижний Новгород,NaN,NaN,NaN


#### 5. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [841]:
# ВАШ КОД ЗДЕСЬ
def vacancy(data, salary_from = False, salary_to = False, test = 0, text = False, skill = False):
    
    '''
    на вход принимает 
    1. data - данные по вакансиями (обязательный параметр)
    
    2. salary_from - нижний порог зарплаты (опционально)
    
    3. salary_to - верхний порог зарплаты (опционально)
    
    4. test - True или False (опционально)
    
    5. text - заданный текст в наименовании вакансии, нерегистрозависимый, передавать в кавычках через запятую без пробелов (опционально)
    
    6. skill - заданный навык (опционально). Если требуется несколько, перечислите их через запятую без пробелов.
    Наличие или всех навыков, или какого-то среза из заданного списка.
    
    '''
    
    
    
    vacancy_df = data
    
    test = test
    
    salary_from = float(salary_from)
    
    salary_to = float(salary_to)
    
    vacancy_text = text
    
    skills = skill
    
    # поработаем с типом данных
    vacancy_df['created'] = pd.to_datetime(vacancy_df['created'])
    vacancy_df['created'] = vacancy_df['created'].dt.strftime('%Y-%m-%d')
    vacancy_df['salary_from'] = vacancy_df['salary_from'].astype(float)
    vacancy_df['salary_to'] = vacancy_df['salary_to'].astype(float)
    
    # Пусть функция принимает значения в рублях (можно сделать и с выбором валюты, но в условии этого не задано)
    # Переведём валюты в рубли
    
    # возьмём курс $ на сегодняшнюю дату
    usd_rate = float(
                    ET.fromstring(requests.get('http://www.cbr.ru/scripts/XML_daily.asp').text)
                    .find("./Valute[CharCode='USD']/Value")
                    .text.replace(",","."))


    # выполним запрос к апи ЦБ РФ
    # возьмём курс € на сегодняшнюю дату
    euro_rate = float(
                    ET.fromstring(requests.get('http://www.cbr.ru/scripts/XML_daily.asp').text)
                    .find("./Valute[CharCode='EUR']/Value")
                    .text.replace(",","."))


    # приведём валюты к рублю
    for i in range(len(df)):

        if vacancy_df['currency'][i] == "USD":

            vacancy_df['currency'][i] = "RUR"

            vacancy_df['salary_to'][i]  = df['salary_to'][i]*usd_rate
            vacancy_df['salary_from'][i]  = df['salary_from'][i]*usd_rate

        elif vacancy_df['currency'][i] == "EUR":

            vacancy_df['currency'][i] = "RUR"

            vacancy_df['salary_to'][i]  = df['salary_to'][i]*euro_rate
            vacancy_df['salary_from'][i]  = df['salary_from'][i]*euro_rate

        else:
            pass
        
    if salary_from != False:
        
        #значение минимальной заработной платы
        vacancy_df = vacancy_df.query('salary_from >= @salary_from')
        
    else:
        pass
    
    if salary_to != False:
        
        #значение максимальной заработной платы
        vacancy_df = vacancy_df.query('salary_to <= @salary_to')
        
    else:
        pass
    
    # наличие тестового задания
    if test == True:
        
        vacancy_df = vacancy_df.query('has_test == True')
        
    elif test == False:
        
        vacancy_df = vacancy_df.query('has_test == False')
        
    else:
        pass
    
    # наличие заданного текста в вакансии
    if vacancy_text != False:
        
        # требуемые слова
        words = vacancy_text.split(',')
        vacancy_pattern = '|'.join(words)
        
        # ищем слова
        vacancy_df = vacancy_df[vacancy_df['vacancy'].str.contains(vacancy_pattern, case=False, regex = True)]
        
    else:
        pass

    # наличие заданного навыка (-ов)
    if skills != False:
        
        skill_words = skills.split(',')
        skill_pattern = '|'.join(skill_words)
        vacancy_df = vacancy_df[vacancy_df['skills'].str.contains(skill_pattern, case=False, na = False)]
        
    else:
        pass
    
    # количество вакансий
    vacancies_number = str(vacancy_df.shape[0])
    
    print("Вакансий найдено: " + vacancies_number)
    
    # в среднем зп начинается от:
    mean_salary_from = str(vacancy_df['salary_from'].mean())
    
    print("Заработная плата в среднем начинается от " + mean_salary_from)
    
    # ссылки на вакансии
    print("Ссылки на вакансии по запросу:")
          
    for x in vacancy_df['url']:
        print(str(x))
    
    return(vacancy_df)

In [842]:
print(vacancy.__doc__)


    на вход принимает 
    1. data - данные по вакансиями (обязательный параметр)
    
    2. salary_from - нижний порог зарплаты (опционально)
    
    3. salary_to - верхний порог зарплаты (опционально)
    
    4. test - True или False (опционально)
    
    5. text - заданный текст в наименовании вакансии, нерегистрозависимый, передавать в кавычках через запятую без пробелов (опционально)
    
    6. skill - заданный навык (опционально). Если требуется несколько, перечислите их через запятую без пробелов.
    Наличие или всех навыков, или какого-то среза из заданного списка.
    
    


#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [843]:
# ВАШ КОД ЗДЕСЬ

In [844]:
df = pd.read_csv('df2021-08-03.csv')
vacancy_df = vacancy(df,0, 20000000, test = False, text = "python,senior", skill = "PHP,sql")


Вакансий найдено: 226
Заработная плата в среднем начинается от 123070.38581005586
Ссылки на вакансии по запросу:
https://hh.ru/applicant/vacancy_response?vacancyId=46030221
https://hh.ru/applicant/vacancy_response?vacancyId=42905077
https://hh.ru/applicant/vacancy_response?vacancyId=46614391
https://hh.ru/applicant/vacancy_response?vacancyId=46575862
https://hh.ru/applicant/vacancy_response?vacancyId=45924294
https://hh.ru/applicant/vacancy_response?vacancyId=46386511
https://hh.ru/applicant/vacancy_response?vacancyId=46325809
https://hh.ru/applicant/vacancy_response?vacancyId=37029603
https://hh.ru/applicant/vacancy_response?vacancyId=43674070
https://hh.ru/applicant/vacancy_response?vacancyId=45797134
https://hh.ru/applicant/vacancy_response?vacancyId=46089335
https://hh.ru/applicant/vacancy_response?vacancyId=43674066
https://hh.ru/applicant/vacancy_response?vacancyId=46141622
https://hh.ru/applicant/vacancy_response?vacancyId=46173267
https://hh.ru/applicant/vacancy_response?vacanc